In [1]:

%load_ext autoreload
%autoreload 2
import json, os, cv2
from pathlib import Path
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import xarray as xr
import seaborn as sns
import h5py as hf
from tqdm import tqdm
from tqdm import tqdm
import itertools 
from scipy.interpolate import interp1d
from scipy import signal
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors
import os, fnmatch
from scipy.spatial.distance import cdist
import matplotlib.gridspec as gridspec
from random import sample
from scipy.ndimage import gaussian_filter1d
from matplotlib.collections import LineCollection


import sys
sys.path.insert(0, 'C:/Users/nlab/Documents/GitHub/obstacle_avoidance')

from utils.base_functions import *
from src.utils.auxiliary import flatten_series
from src.utils.path import find
from src.base import BaseInput
from plots.plots import plot_oa


import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
c:\Users\nlab\anaconda3\envs\ephys0\lib\site-packages\deeplabcut\__init__.py:78: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


In [2]:
df = pd.read_hdf(r"D:\obstacle_avoidance\recordings\061923\G8CK1LN\oadark\test_G8CK1LN_061923_oadark.h5")

In [3]:
df

,first_poke,second_poke,trial_timestamps,trial_vidframes,nose_x,nose_y,nose_likelihood,leftear_x,leftear_y,leftear_likelihood,...,gt_obstacleTR_x_cm,gt_obstacleTR_y_cm,gt_obstacleBR_x_cm,gt_obstacleBR_y_cm,gt_obstacleBL_x_cm,gt_obstacleBL_y_cm,gt_obstacle_cen_x,gt_obstacle_cen_x_cm,gt_obstacle_cen_y,gt_obstacle_cen_y_cm
0,47340.634739,47343.240025,"[47340.645478, 47340.662809, 47340.678886, 473...","[276, 277, 278, 279, 280, 281, 282, 283, 284, ...","[109.85092163085938, 110.42352294921875, 110.5...","[271.0501403808594, 270.61016845703125, 269.96...","[0.974859893321991, 0.9687406420707703, 0.9718...","[125.8490982055664, 124.87584686279297, 125.22...","[266.9076232910156, 266.72589111328125, 266.76...","[0.99748694896698, 0.9972206354141235, 0.99724...",...,27.524722,8.074960,27.874296,22.172279,23.132217,22.326554,321.490988,25.261812,193.398440,15.196678
1,47343.240025,47346.656396,"[47343.246528, 47343.263219, 47343.280512, 473...","[[432, 433, 434, 435, 436, 437, 438, 439, 440,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.26832515001296997, 0.2303224503993988, 0.14...","[673.2642211914062, 674.18115234375, 675.25091...","[256.1656188964844, 255.76043701171875, 255.75...","[0.9983585476875305, 0.998097836971283, 0.9988...",...,27.501998,8.069367,27.857385,22.102597,23.186248,22.306025,321.529762,25.264858,193.092154,15.172611
2,47346.656396,47348.786393,"[47346.664076, 47346.680883, 47346.697753, 473...","[637, 638, 639, 640, 641, 642, 643, 644, 645, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.02296832576394081, 0.022547217085957527, 0....","[117.62759399414062, 116.33463287353516, 115.4...","[264.8180236816406, 264.5900573730469, 264.782...","[0.9535960555076599, 0.9607521891593933, 0.975...",...,27.521476,8.073352,27.808827,22.068261,23.086874,22.273331,321.130239,25.233465,192.897752,15.157335
3,47348.786393,47351.223782,"[47348.798195, 47348.815488, 47348.83182, 4734...","[[765, 766, 767, 768, 769, 770, 771, 772, 773,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.1938612461090088, 0.1826181560754776, 0.154...","[675.5137939453125, 675.9728393554688, 676.655...","[257.1358642578125, 257.03826904296875, 257.29...","[0.9992577433586121, 0.9994097948074341, 0.999...",...,27.523250,8.075104,27.890333,22.079419,23.161543,22.287590,321.623301,25.272208,192.980109,15.163807
5,47361.276185,47363.385843,"[47361.285734, 47361.302656, 47361.319308, 473...","[[1514, 1515, 1516, 1517, 1518, 1519, 1520, 15...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.3845151960849762, 0.3125222623348236, 0.240...","[673.2482299804688, 673.2340087890625, 673.972...","[257.9873962402344, 258.1358642578125, 258.216...","[0.9993182420730591, 0.999315619468689, 0.9994...",...,38.080485,13.019589,38.059412,26.851090,33.410447,26.837535,453.777342,35.656482,253.983499,19.957273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,47872.274508,47874.822387,"[47872.279052, 47872.29582, 47872.31264, 47872...","[[32163, 32164, 32165, 32166, 32167, 32168, 32...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.10890915244817734, 0.07977021485567093, 0.0...","[680.031982421875, 681.1895141601562, 682.7614...","[248.99256896972656, 249.90663146972656, 252.9...","[0.9969045519828796, 0.9967404007911682, 0.996...",...,40.065819,20.161896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,47874.822387,47876.428236,"[47874.829977, 47874.847091, 47874.863552, 478...","[32316, 32317, 32318, 32319, 32320, 32321, 323...","[110.65408325195312, 112.1395492553711, 112.02...","[271.0017395019531, 270.81024169921875, 270.82...","[0.9227438569068909, 0.8722624182701111, 0.762...","[125.85198974609375, 124.10636138916016, 123.4...",

In [4]:
time_thresh = df['len'].quantile(0.9)
df['time'] = df['len']/60
df = df[df['len']<time_thresh]
session = plot_oa('empty',df)
session.single_day_df('obstacle',6)
session.plot_trace_cluster_single_animal()

KeyError: 'start'